In [2]:
# 노션 논문 참고, 여러 모델 중 가장 성능 좋은 GRU 모델 이용
# input은 가장 관련도 높은 close, high, PER, foreign_holding 이용
# 하이퍼파라미터 최적화 -> optimizer는 adam, batch_size는 64, hidden layer는 3개

# 기본 OHLCV 데이터 + 기술적 지표 적용 + 재무제표 분석 적용한 데이터셋(csv파일)을 your_data라 했을 때의 코드

### train, test 과정

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

# 데이터 로드
data = pd.read_csv("your_data.csv")

# 데이터 전처리
data['date'] = pd.to_datetime(data['date'])
data = data.sort_values('date')
scaler = MinMaxScaler()
data[['close', 'high', 'PER', 'foreign_holding']] = scaler.fit_transform(data[['close', 'high', 'PER', 'foreign_holding']])

# 학습/테스트 데이터 분할
train_data = data[(data['date'] >= '2019-01-01') & (data['date'] <= '2023-06-30')]
test_data = data[(data['date'] >= '2023-07-01') & (data['date'] <= '2023-12-31')]

X_train = []
y_train = []
X_test = []
y_test = []

# 시계열 데이터 준비 (15일을 기준으로 입력 데이터 생성)
window_size = 15

for i in range(window_size, len(train_data)):
    X_train.append(train_data[['high', 'PER', 'foreign_holding']].values[i-window_size:i])
    y_train.append(train_data['close'].values[i])

for i in range(window_size, len(test_data)):
    X_test.append(test_data[['high', 'PER', 'foreign_holding']].values[i-window_size:i])
    y_test.append(test_data['close'].values[i])

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)

# 모델 정의
model = Sequential()
model.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(GRU(units=50, return_sequences=True))
model.add(GRU(units=50))
model.add(Dense(1))

# 모델 컴파일
model.compile(optimizer=Adam(), loss='mean_squared_error')

# 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# 예측
predicted_prices = model.predict(X_test)
predicted_prices = scaler.inverse_transform(np.concatenate((predicted_prices, np.zeros((predicted_prices.shape[0], 3))), axis=1))[:, 0]
y_test_actual = scaler.inverse_transform(np.concatenate((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], 3))), axis=1))[:, 0]

# 예측 결과와 실제 값 비교
predicted_vs_actual = pd.DataFrame({'Actual': y_test_actual, 'Predicted': predicted_prices})
print(predicted_vs_actual.head())


### 새로운 날짜에 대한 주식 가격 예측

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

# 데이터 로드
data = pd.read_csv("your_data.csv")

# 데이터 전처리
data['date'] = pd.to_datetime(data['date'])
data = data.sort_values('date')
scaler = MinMaxScaler()
data[['close', 'high', 'PER', 'foreign_holding']] = scaler.fit_transform(data[['close', 'high', 'PER', 'foreign_holding']])

# 학습 데이터 준비 (2019년부터 2023년까지)
train_data = data[(data['date'] >= '2019-01-01') & (data['date'] <= '2023-12-31')]

X_train = []
y_train = []
window_size = 15

for i in range(window_size, len(train_data)):
    X_train.append(train_data[['high', 'PER', 'foreign_holding']].values[i-window_size:i])
    y_train.append(train_data['close'].values[i])

X_train, y_train = np.array(X_train), np.array(y_train)

# 모델 정의
model = Sequential()
model.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(GRU(units=50, return_sequences=True))
model.add(GRU(units=50))
model.add(Dense(1))

# 모델 컴파일
model.compile(optimizer=Adam(), loss='mean_squared_error')

# 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=64)

# 2024년 예측 준비
# 마지막 15일 데이터로부터 2024년 데이터를 순차적으로 예측
predicted_prices = []
last_window = train_data[['high', 'PER', 'foreign_holding']].values[-window_size:]

for day in range(365):  # 2024년 예측일수 (영업일 고려하여 조정 가능)
    x_input = last_window.reshape((1, window_size, 3))
    predicted_price = model.predict(x_input)
    
    predicted_prices.append(predicted_price[0][0])
    
    # 예측한 값을 윈도우에 추가하여 다음 예측에 사용
    last_window = np.append(last_window[1:], [[predicted_price[0][0], last_window[-1][1], last_window[-1][2]]], axis=0)

# 예측 결과 스케일 원상 복구
predicted_prices = scaler.inverse_transform(np.concatenate((np.array(predicted_prices).reshape(-1, 1), 
                                                           np.zeros((365, 3))), axis=1))[:, 0]

# 예측 결과 출력
predicted_2024 = pd.DataFrame({
    'Date': pd.date_range(start='2024-01-01', periods=365, freq='B'),  # 주말 및 휴일 제외한 영업일 기준
    'Predicted_Close': predicted_prices
})
print(predicted_2024.head())
